In [80]:
import os
import uuid
import zipfile
from os.path import join, dirname
from tempfile import gettempdir
import numpy as np

from aequilibrae import TrafficAssignment, TrafficClass, Graph, Project
from tests.data import siouxfalls_project

In [81]:
# Set up data:
os.environ["PATH"] = os.path.join(gettempdir(), "temp_data") + ";" + os.environ["PATH"]
proj_path = os.path.join(gettempdir(), "test_odme_files" + uuid.uuid4().hex)
os.mkdir(proj_path)
zipfile.ZipFile(join(dirname(siouxfalls_project), "sioux_falls_single_class.zip")).extractall(proj_path)

# Initialise project:
project = Project()
project.open(proj_path)
project.network.build_graphs()
car_graph = project.network.graphs["c"]  # type: Graph

car_graph.set_graph("free_flow_time")
car_graph.set_blocked_centroid_flows(False)
matrix = project.matrices.get_matrix("demand_omx")
matrix.computational_view()

# Initial assignment parameters (with conjugate Frank-Wolfe):
assignment = TrafficAssignment()
assignclass = TrafficClass("car", car_graph, matrix)
assignment.set_classes([assignclass])
assignment.set_vdf("BPR")
assignment.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
assignment.set_vdf_parameters({"alpha": "b", "beta": "power"})
assignment.set_capacity_field("capacity")
assignment.set_time_field("free_flow_time")
assignment.max_iter = 5
assignment.set_algorithm("cfw")

In [82]:
assignclass.set_select_links({"sl_1_1": [(1, 1)], "sl_5_1": [(5, 1)]})
assignment.execute()
matrix.matrix_view = np.squeeze(matrix.matrix_view, axis=2)

In [83]:
assign_df = assignment.results().reset_index(drop=False).fillna(0)
sl_matrix_1 = assignclass.results.select_link_od.matrix["sl_1_1"].squeeze()
sl_matrix_5 = assignclass.results.select_link_od.matrix["sl_5_1"].squeeze()

In [84]:
assigned_flow_1 = assign_df.loc[assign_df["link_id"] == 1, "matrix_ab"].values[0]
assigned_flow_5 = assign_df.loc[assign_df["link_id"] == 5, "matrix_ab"].values[0]
sl_flow_1 = np.sum(sl_matrix_1)
sl_flow_5 = np.sum(sl_matrix_5)

In [85]:
print("Assigned 1: ", assigned_flow_1)
print("Select Link 1: ", sl_flow_1)

print("Assigned 5: ", assigned_flow_5)
print("Select Link 5: ", sl_flow_5)

Assigned 1:  6971.29633565599
Select Link 1:  5878.615489421606
Assigned 5:  10627.825456440407
Select Link 5:  9535.144610206024


In [86]:
assert assigned_flow_1 == sl_flow_1
assert assigned_flow_5 == sl_flow_5

AssertionError: 